## Maximal Update Parameterization

In this problem, we will examine the training of a simple MLP with hidden layers of varying widths. We will then investigate the maximal update parameterization (muP) which will allow us to use a single global learning rate to jointly train layers of any width.

Note: This homework question is new this year and it is messier than usual. We felt it was worth it to get it out so you can play with these new techniques. If you're feeling stuck, don't hesistate to ask questions on Ed.


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from keras.datasets import mnist
import matplotlib.pyplot as plt

%matplotlib inline

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

class MLP(nn.Module):
    def __init__(self, input_size=784, hidden_sizes = [8, 16, 32, 64, 128], num_classes=10):
        super(MLP, self).__init__()
        all_hidden_sizes = [input_size] + hidden_sizes + [num_classes]
        self.layers = nn.ModuleList()
        for i in range(len(all_hidden_sizes)-1):
            self.layers.append(nn.Linear(all_hidden_sizes[i], all_hidden_sizes[i+1]))
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        activations = []
        x = x.view(x.size(0), -1)  # Flatten: (batch_size, 28*28)
        for layer in self.layers[:-1]:
            x = self.sigmoid(layer(x))
            activations.append(x)
        x = self.layers[-1](x)
        activations = activations[1:]
        return x, [a.detach() for a in activations]

# Load MNIST data
(train_images, train_labels), (valid_images, valid_labels) = mnist.load_data()

# Normalize pixel values to [0, 1]
train_images = train_images.astype(np.float32) / 255.0
valid_images = valid_images.astype(np.float32) / 255.0

# Convert to PyTorch tensors
train_images = torch.from_numpy(train_images)
train_labels = torch.from_numpy(train_labels).long()
valid_images = torch.from_numpy(valid_images)
valid_labels = torch.from_numpy(valid_labels).long()

def rms(x, dim):
    return torch.sqrt(torch.mean(x**2, dim=dim))


In [ ]:
from torch.optim.optimizer import Optimizer
from typing import Any
class SimpleAdam(Optimizer):
    def __init__(
        self,
        params: Any,
        lr: float = 1e-1,
        b1: float = 0.9,
        b2: float = 0.999,
    ):
        defaults = dict(lr=lr, b1=b1, b2=b2,)
        super(SimpleAdam, self).__init__(params, defaults)

    @torch.no_grad()
    def step(self):
        for group in self.param_groups:
            for p in group['params']:
                grad = p.grad.data

                state = self.state[p]
                if len(state) == 0: # Initialization
                    state["step"] = torch.tensor(0.0)
                    state['momentum'] = torch.zeros_like(p)
                    state['variance'] = torch.zeros_like(p)

                state['step'] += 1
                m = state['momentum']
                m.lerp_(grad, 1-group["b1"])
                v = state['variance']
                v.lerp_(grad**2, 1-group["b2"])

                m_hat = m / (1 - group["b1"]**state['step'])
                v_hat = v / (1 - group["b2"]**state['step'])
                u = m_hat / (torch.sqrt(v_hat) + 1e-16)

                p.add_(u, alpha=-group['lr'])
        return None

In [ ]:
batch_idx = np.random.randint(0, len(train_images), size=64)
def train_one_step(mlp=MLP, hiddens=[8, 16, 64, 64, 64, 256, 256, 1024], optimizer=SimpleAdam, label="Adam", lr=0.01):
    model = mlp(hidden_sizes=hiddens).to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optimizer(model.parameters(), lr=lr)

    prev_activations = None
    for i in range(2):
        images_batch = train_images[batch_idx]
        labels_batch = train_labels[batch_idx]
        images_batch, labels_batch = images_batch.to(device), labels_batch.to(device)

        optimizer.zero_grad()
        outputs, activations = model(images_batch)
        loss = criterion(outputs, labels_batch)
        loss.backward()
        optimizer.step()

        if i > 0:
            print([a.shape for a in activations])
            activation_deltas = [a - pa for a, pa in zip(activations, prev_activations)]
            activation_deltas_rms = [torch.mean(rms(a, dim=-1)) for a in activation_deltas]
        prev_activations = activations

    # plot deltas
    deltas = np.array(activation_deltas_rms)
    fig, axs = plt.subplots(1, figsize=(8, 4))
    axs.set_title(f'RMS of activation deltas per layer ({label})')
    axs.set_xlabel('Hidden Size of activation')
    axs.bar(np.arange(deltas.shape[0]), deltas)
    axs.set_xticks(np.arange(deltas.shape[0]))
    axs.set_xticklabels(hiddens[1:])
    plt.show()
train_one_step(optimizer=SimpleAdam)

## a. Examining the norms of a heterogenous MLP.

Run the above cell, which trains a neural network for a single gradient step, then examines the effect of that step on the resulting activations. What are the dimensions of each layer in the neural network? 

*Answer:* 

How does the dimensionality of the layer affect the RMS norm of the activation deltas?

*Answer:*

Change the widths of some of your neural network layers, and recreate the plot -- did the RMS values change as expected?

*Answer:*


In [ ]:
# TODO: Call some plotting code here.

## b. Examining the norms of the updates to the weights.

In the provided code above, we plotted the change in norms of the *activation vectors*. Now, you will examine the change in the weights themselves. Create a version of the above function that runs a single gradient step, then for each dense layer plot:
- The *Frobenius* norm of the update.
- The *spectral* norm of the update.
- The *RMS-RMS induced norm* of the update.

Which one of these norms correlates the most with the RMS norms of the activations?

*Answer:*

You should calculate your updates as `new_dense_parameter - old_dense_parameter`.

In [ ]:
### Solution
batch_idx = np.random.randint(0, len(train_images), size=64)
def train_one_step_matrices(mlp=MLP, hiddens=[8, 16, 64, 64, 64, 256, 256, 1024], optimizer=SimpleAdam, label="Adam", lr=0.01):
    model = mlp(hidden_sizes=hiddens).to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optimizer(model.parameters(), lr=lr)

    old_params = [p.detach().clone() for p in model.parameters()]

    for i in range(1):
        images_batch = train_images[batch_idx]
        labels_batch = train_labels[batch_idx]
        images_batch, labels_batch = images_batch.to(device), labels_batch.to(device)

        optimizer.zero_grad()
        outputs, activations = model(images_batch)
        loss = criterion(outputs, labels_batch)
        loss.backward()
        optimizer.step()

    new_params = [p.detach().clone() for p in model.parameters()]
    delta_params = [new_p - old_p for new_p, old_p in zip(new_params, old_params)]

    frob_norms = []
    spectral_norms = []
    induced_norms = []
    p_shapes = []

    for p in delta_params:
        if len(p.shape) == 2:
            ### TODO: Log the respective norms.
            pass
            ###


    fig, axs = plt.subplots(1, 3, figsize=(20, 6))

    axs[0].set_title(f'Frobenius norm of update per layer ({label})')
    axs[0].set_xlabel('Hidden Size of layer')
    axs[0].bar(np.arange(len(frob_norms)), frob_norms)
    axs[0].set_xticks(np.arange(len(frob_norms)), p_shapes, rotation=45, ha='right')

    axs[1].set_title(f'Spectral norm of update per layer ({label})')
    axs[1].set_xlabel('Hidden Size of layer')
    axs[1].bar(np.arange(len(spectral_norms)), spectral_norms)
    axs[1].set_xticks(np.arange(len(spectral_norms)), p_shapes, rotation=45, ha='right')

    axs[2].set_title(f'Induced norm of update per layer ({label})')
    axs[2].set_xlabel('Hidden Size of layer')
    axs[2].bar(np.arange(len(induced_norms)), induced_norms)
    axs[2].set_xticks(np.arange(len(induced_norms)), p_shapes, rotation=45, ha='right')
    plt.show()
train_one_step_matrices(optimizer=SimpleAdam)

## c. Lion: A Sign-based Optimizer

In this section, you will implement a simplified **Lion**-style optimizer and compare its behavior to `SimpleAdam` on the same MLP setup as above.

The full Lion optimizer (Chen et al., 2023) is described in the paper:

- Lion: Sign-based optimizer with memory. [[arXiv:2302.06675](https://arxiv.org/abs/2302.06675)]

In our simplified version, we keep an exponential-moving-average momentum buffer
$\mathbf{m}_t$ and use only its **sign** to form the update:

$$
\mathbf{m}_t = \beta_1 \mathbf{m}_{t-1} + (1 - \beta_1)\, \mathbf{g}_t,\\
\theta_{t+1} = \theta_t - \eta \, \operatorname{sign}(\mathbf{m}_t),
$$

where $\mathbf{g}_t$ is the gradient at step $t$, $\beta_1$ is the momentum
parameter, and $\eta$ is the learning rate.

We will:

1. Implement a `SimpleLion` optimizer class, similar to `SimpleAdam`.
2. Use `train_one_step` to compare Lion and Adam in terms of RMS activation deltas per layer.
3. Perform a small hyperparameter sweep over Lion's learning rate and momentum using the provided code.

Below, fill in the TODOs in the `SimpleLion` class. Then follow the instructions in the subsequent markdown cells to run the experiments.

In [ ]:
from torch.optim.optimizer import Optimizer
from typing import Any

class SimpleLion(Optimizer):
    """Simplified Lion-style optimizer.

    This version keeps an exponential-moving-average momentum buffer m
    and applies sign-based updates using that momentum.
    """
    def __init__(
        self,
        params: Any,
        lr: float = 1e-1,
        b1: float = 0.9,
    ):
        defaults = dict(lr=lr, b1=b1)
        super(SimpleLion, self).__init__(params, defaults)

    @torch.no_grad()
    def step(self):
        for group in self.param_groups:
            lr = group["lr"]
            b1 = group["b1"]
            for p in group["params"]:
                if p.grad is None:
                    continue
                grad = p.grad.data

                state = self.state[p]
                if len(state) == 0:  # initialization
                    state["momentum"] = torch.zeros_like(p)

                m = state["momentum"]
                ################################################################
                # TODO: Implement the Lion-style step in three steps:           #
                #   1) Update the momentum buffer m using grad and b1.         #
                #   2) Compute an update tensor u based on sign(m).            #
                #   3) Apply u to p with learning rate lr.                     #
                ################################################################
                # YOUR CODE HERE                                               #
                ################################################################
                pass
        return None

### c.1 Comparing activation deltas for Adam vs Lion

Once you have implemented `SimpleLion`, compare how **activation deltas** behave under Adam vs Lion.

1. Re-run `train_one_step` with `optimizer=SimpleLion` and the same hidden sizes and learning rate as the Adam experiment:

   ```python
   train_one_step(optimizer=SimpleLion, label="Lion", lr=0.01)
   ```

2. On the same figure (or side-by-side), compare the RMS of activation deltas per layer for:

   - `SimpleAdam` (from part a)
   - `SimpleLion` (this part)

In your written submission, briefly answer:

- Which layers show the largest change under Lion vs Adam?
- Does Lion tend to make activation deltas more uniform or more peaked across layers?
- How might this behavior be related to using only the **sign** of the momentum in the update?

### c.2 Hyperparameter sweep for Lion

Next, run the **hyperparameter sweep** for Lion using the code cell directly below.

- The sweep considers learning rates `lr` in `[0.001, 0.003, 0.01, 0.03]` and momentum parameters `b1` in `[0.8, 0.9, 0.95]` for a 3-layer MLP of width 64.
- The code will report the mean validation loss for each `(lr, b1)` pair and plot the results.

In your written submission, report one reasonably good `(lr, b1)` pair and comment (very briefly) on how sensitive Lion seems to be to the learning rate and momentum in this setting.

In [ ]:
# Hyperparameter sweep for Lion

# This cell sweeps over lr and b1 for SimpleLion on a 3-layer MLP of width 64,
# reports the mean validation loss, and plots the results.

def train_with_lr_lion(hiddens=[64, 64, 64], lr=0.01, b1=0.9):
    """Train a 3-layer MLP with SimpleLion and return mean valid loss.

    This mirrors `train_with_lr` but exposes both lr and b1.
    """
    torch.manual_seed(4)
    np.random_seed(4)
    model = MLP(hidden_sizes=hiddens).to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = SimpleLion(model.parameters(), lr=lr, b1=b1)
    losses = []

    for i in range(100):
        batch_idx = np.random.randint(0, len(train_images), size=64)

        images_batch = train_images[batch_idx]
        labels_batch = train_labels[batch_idx]
        images_batch, labels_batch = images_batch.to(device), labels_batch.to(device)

        optimizer.zero_grad()
        outputs, _ = model(images_batch)
        loss = criterion(outputs, labels_batch)
        loss.backward()
        optimizer.step()

        with torch.no_grad():
            outputs_valid, _ = model(valid_images)
            valid_losses = criterion(outputs_valid, valid_labels)
            losses.append(valid_losses.item())

    return np.mean(np.array(losses)[-5:])


lrs = [0.001, 0.003, 0.01, 0.03]
b1s = [0.8, 0.9, 0.95]
results = np.zeros((len(b1s), len(lrs)))

for i, b1 in enumerate(b1s):
    for j, lr in enumerate(lrs):
        results[i, j] = train_with_lr_lion(hiddens=[64, 64, 64], lr=lr, b1=b1)

print("Mean validation loss for each (b1, lr):")
for i, b1 in enumerate(b1s):
    for j, lr in enumerate(lrs):
        print(f"b1={b1}, lr={lr}: loss={results[i, j]:.3f}")

# Plot results: loss vs lr, one curve per b1
fig, ax = plt.subplots(1, figsize=(8, 4))
for i, b1 in enumerate(b1s):
    ax.plot(np.arange(len(lrs)), results[i], marker='o', label=f"b1={b1}")
ax.set_title("Lion: mean validation loss vs learning rate")
ax.set_xlabel("learning rate")
ax.set_ylabel("mean validation loss")
ax.set_xticks(np.arange(len(lrs)))
ax.set_xticklabels(lrs)
ax.legend()
plt.show()

## d. Implementing muP

We will now implement muP scaling. Modify the starter code below to set a per-layer learning rate such that the resulting RMS activation-deltas are uniform scale, regardless of the layer widths. Plot the resulting activation-deltas on at least two sets of widths.

Note: Even with the correct scaling, the first 2-3 activation-deltas may have a lower norm than the rest. Can you think of a reason why this might be the case?

In [ ]:
from torch.optim.optimizer import Optimizer
from typing import Any
class SimpleAdamMuP(Optimizer):
    def __init__(
        self,
        params: Any,
        lr: float = 1e-1,
        b1: float = 0.9,
        b2: float = 0.999,
    ):
        defaults = dict(lr=lr, b1=b1, b2=b2,)
        super(SimpleAdamMuP, self).__init__(params, defaults)

    @torch.no_grad()
    def step(self):
        for group in self.param_groups:
            for p in group['params']:
                grad = p.grad.data

                state = self.state[p]
                if len(state) == 0: # Initialization
                    state["step"] = torch.tensor(0.0)
                    state['momentum'] = torch.zeros_like(p)
                    state['variance'] = torch.zeros_like(p)

                state['step'] += 1
                m = state['momentum']
                m.lerp_(grad, 1-group["b1"])
                v = state['variance']
                v.lerp_(grad**2, 1-group["b2"])

                m_hat = m / (1 - group["b1"]**state['step'])
                v_hat = v / (1 - group["b2"]**state['step'])
                u = m_hat / (torch.sqrt(v_hat) + 1e-16)

                ############################
                ### Todo: Adjust the per-layer learning rate scaling factor so per-layer RMS activation deltas are constant.
                ### Hint for part f: The following tricks will help you retain performance when using muP scaling.
                ###  - Treat biases as a hidden layer with size (d_out, 1). You will need to use a fudge-factor of around 0.01 -- we want to keep the change in bias terms low.
                ###  - For the input layer, a fudge factor of 10 appears to help.
                ###  - For the output layer, we find it is best to ignore the muP scaling, and instead use a fixed learning rate (e.g. 0.003).
                ############################
                lr = group['lr']
                pass
                ############################
                ############################

                p.add_(u, alpha=-lr)
        return None
train_one_step(optimizer=SimpleAdamMuP, lr=2, label="Adam MuP", hiddens=[8, 16, 64, 64, 64, 256, 256, 1024])
train_one_step(optimizer=SimpleAdamMuP, lr=2, label="Adam MuP", hiddens=[8, 16, 32, 64, 128, 256, 512, 1024])

## e. Per-Weight Multipliers

An alternative way to implement muP is to adjust the *network graph* itself, rather than the optimizer. Implement this below, and recreate the above uniformly-scaled graph when using the *Adam* (not muP) optimizer. We have disabled biases to simplify the problem.

Why is multiplying the output of a layer by a constant the same as adjusting the learning-rate of that layer (when using Adam or SignGD)?

*Answer:*

In [ ]:
class ScaledMLP(nn.Module):
    def __init__(self, input_size=784, hidden_sizes = [8, 16, 32, 64, 128], num_classes=10):
        super(ScaledMLP, self).__init__()
        all_hidden_sizes = [input_size] + hidden_sizes + [num_classes]
        self.layers = nn.ModuleList()
        for i in range(len(all_hidden_sizes)-1):
            self.layers.append(nn.Linear(all_hidden_sizes[i], all_hidden_sizes[i+1], bias=False))
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        activations = []
        x = x.view(x.size(0), -1)  # Flatten: (batch_size, 28*28)
        for layer in self.layers[:-1]:
            x = layer(x)
            ## TODO
            pass
            ##
            x = self.sigmoid(x)
            activations.append(x)
        x = self.layers[-1](x)
        activations = activations[1:]
        return x, [a.detach() for a in activations]

train_one_step(mlp=ScaledMLP, optimizer=SimpleAdam)

## f. Hyperparameter Transfer

Run the following code, which will perform a sweep over learning rates for 3-layer MLPs of increasing width using Adam. How does the optimal learning rate change as the network increases in size?

*Answer:*

In the second cell, we will instead use the muP optimizer you implemented. How does the optimal learning rate work now? You should aim to show that there is a single global learning rate that works on a majority of widths. The 256-width network should achieve a loss of 0.5, comparable to Adam.

*Answer:*

In [ ]:
valid_idx = np.random.randint(0, len(train_images), size=64)
valid_images = train_images[valid_idx]
valid_labels = train_labels[valid_idx]
valid_images, valid_labels = valid_images.to(device), valid_labels.to(device)


def train_with_lr(hiddens=[64, 64, 64], optimizer=SimpleAdam, lr=0.01):
    torch.manual_seed(4)
    np.random.seed(4)
    model = MLP(hidden_sizes=hiddens).to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optimizer(model.parameters(), lr=lr)
    losses = []

    for i in range(100):
        batch_idx = np.random.randint(0, len(train_images), size=64)

        images_batch = train_images[batch_idx]
        labels_batch = train_labels[batch_idx]
        images_batch, labels_batch = images_batch.to(device), labels_batch.to(device)

        optimizer.zero_grad()
        outputs, _ = model(images_batch)
        loss = criterion(outputs, labels_batch)
        loss.backward()
        optimizer.step()

        with torch.no_grad():
            outputs_valid, _ = model(valid_images)
            valid_losses = criterion(outputs_valid, valid_labels)
            losses.append(valid_losses.item())

    return np.mean(np.array(losses)[-5:])



In [ ]:
all_widths = [4, 8, 16, 32, 64, 128, 256]
all_lrs = [0.0001, 0.0003, 0.001, 0.003, 0.01, 0.03, 0.1, 0.3, 1.0]
adam_results = np.zeros((len(all_widths), len(all_lrs)))
for wi, width in enumerate(all_widths):
    for lri, lr in enumerate(all_lrs):
        loss = train_with_lr(hiddens=[width, width, width], lr=lr)
        adam_results[wi, lri] = loss

fig, axs = plt.subplots(1, figsize=(8, 4))
axs.set_title(f'Loss per learning rate (Adam)')
for wi, width in enumerate(all_widths):
    axs.plot(np.arange(len(all_lrs)), adam_results[wi], label=f'Width: {width}')
axs.set_xticks(np.arange(len(all_lrs)))
axs.set_xticklabels(all_lrs)
axs.set_ylim(bottom=0, top=3)
axs.legend()
fig.show()

In [ ]:
all_widths = [4, 8, 16, 32, 64, 128, 256]
all_lrs = [0.03, 0.1, 0.3, 1.0, 3.0, 10.0]
mup_results = np.zeros((len(all_widths), len(all_lrs)))
for wi, width in enumerate(all_widths):
    for lri, lr in enumerate(all_lrs):
        loss = train_with_lr(hiddens=[width, width, width], lr=lr, optimizer=SimpleAdamMuP)
        mup_results[wi, lri] = loss

fig, axs = plt.subplots(1, figsize=(8, 4))
axs.set_title(f'Loss per learning rate (muP)')
for wi, width in enumerate(all_widths):
    axs.plot(np.arange(len(all_lrs)), mup_results[wi], label=f'Width: {width}')
axs.set_xticks(np.arange(len(all_lrs)))
axs.set_xticklabels(all_lrs)
axs.set_ylim(bottom=0, top=3)
axs.legend()
fig.show()

## g. Shampoo and Orthogonalization

In lecture, we discussed a simplified version of the Shampoo update, which can be viewed as *orthogonalizing* the update to a dense layer. In the following code block, implement this simplified Shampoo update:

$$
momentum \rightarrow U \Sigma V^T. \qquad update = UV^T.
$$

Feel free to use linear algebra functions such as `torch.linalg.svd`.

In [ ]:
from torch.optim.optimizer import Optimizer
from typing import Any
class SimpleShampoo(Optimizer):
    def __init__(
        self,
        params: Any,
        lr: float = 1e-1,
        b1: float = 0.9,
    ):
        defaults = dict(lr=lr, b1=b1)
        super(SimpleShampoo, self).__init__(params, defaults)

    @torch.no_grad()
    def step(self):
        for group in self.param_groups:
            for p in group['params']:
                grad = p.grad.data

                state = self.state[p]
                if len(state) == 0: # Initialization
                    state["step"] = torch.tensor(0.0)
                    state['momentum'] = torch.zeros_like(p)

                state['step'] += 1
                m = state['momentum']
                m.lerp_(grad, 1-group["b1"])

                ############ TODO
                if len(m.shape) == 1:
                    u = m # Ignore biases for this question, it's not important.
                else:
                    pass
                #############
                p.add_(u, alpha=-group['lr'])
        return None

Now, we will examine the relationship between the Frobenius norm and the Spectral norm for Adam vs. Shampoo. Plot these norms using your code from part **b**. What relationship do you see? Can you come up for a reason why this makes sense?

*Answer:*

Bonus: How should we scale the Shampoo update so the *induced RMS-RMS norm* is equal? Implement this change.

In [ ]:
train_one_step_matrices(optimizer=SimpleAdam)
train_one_step_matrices(optimizer=SimpleShampoo, label="Shampoo")

## h. SOAP-style Optimizer

SOAP is a matrix-whitening optimizer closely related to Shampoo. For more details,
see for example:

- *Understanding SOAP from the Perspective of Gradient Whitening*. [[arXiv:2509.22938](https://arxiv.org/abs/2509.22938)]

In this part, you will implement a simple **SOAP-style** optimizer that:

1. Orthogonalizes the momentum using SVD, like Shampoo.
2. Rescales the orthogonal update so that its **Frobenius norm matches** that of the original momentum.

Concretely, for a matrix momentum $m$ at step $t$:

$$
 m_t \xrightarrow{\text{SVD}} U_t \Sigma_t V_t^T, \quad
 u_{0,t} = U_t V_t^T, \quad
 u_t = u_{0,t} \cdot \frac{\lVert m_t \rVert_F}{\lVert u_{0,t} \rVert_F}.
$$

Below, fill in the TODOs in the `SimpleSOAP` class. Then you will compare its
update norms to those of Adam and Shampoo using your `train_one_step_matrices` helper.

In [ ]:
from torch.optim.optimizer import Optimizer
from typing import Any

class SimpleSOAP(Optimizer):
    """Simplified SOAP-style optimizer.

    Uses Shampoo-style orthogonalization (via SVD) and then rescales the
    orthogonal update so that its Frobenius norm matches that of the
    original momentum.
    """
    def __init__(
        self,
        params: Any,
        lr: float = 1e-1,
        b1: float = 0.9,
    ):
        defaults = dict(lr=lr, b1=b1)
        super(SimpleSOAP, self).__init__(params, defaults)

    @torch.no_grad()
    def step(self):
        for group in self.param_groups:
            lr = group["lr"]
            b1 = group["b1"]
            for p in group["params"]:
                if p.grad is None:
                    continue
                grad = p.grad.data

                state = self.state[p]
                if len(state) == 0:  # initialization
                    state["step"] = torch.tensor(0.0)
                    state["momentum"] = torch.zeros_like(p)

                state["step"] += 1
                m = state["momentum"]
                m.lerp_(grad, 1 - b1)

                if len(m.shape) == 1:
                    # For biases, just use the momentum directly
                    u = m
                else:
                    ################################################################
                    # TODO: Implement the SOAP-style matrix update:                #
                    #   1) Compute SVD of m                                       #
                    #   2) Form u0 = U @ Vh                                       #
                    #   3) Rescale u0 so that ||u||_F = ||m||_F                   #
                    ################################################################
                    # YOUR CODE HERE                                               #
                    ################################################################
                    pass
                p.add_(u, alpha=-lr)
        return None

### h.1 Comparing update norms for Adam, Shampoo, and SOAP

Use your `train_one_step_matrices` helper to compare update norms across optimizers.

1. Run the following calls (you may want to reuse the same `batch_idx` for fairness):

   ```python
   train_one_step_matrices(optimizer=SimpleAdam, label="Adam")
   train_one_step_matrices(optimizer=SimpleShampoo, label="Shampoo")
   train_one_step_matrices(optimizer=SimpleSOAP, label="SOAP")
   ```

2. For each optimizer, inspect the bar plots of:

   - Frobenius norm of the update per layer.
   - Spectral norm of the update per layer.
   - RMS-RMS induced norm of the update per layer.

In your written submission, briefly discuss:

- How do the **Frobenius norms** of SOAP compare to those of Adam and Shampoo?
- How do the **spectral norms** compare?
- Which optimizer (Adam, Shampoo, SOAP) seems to give update norms that best match the **activation-delta RMS** patterns from part (a)?

### h.2 Small learning-rate comparison for Shampoo vs SOAP

In this part, you will run the cell below, which performs a
small learning-rate sweep for `SimpleShampoo` and `SimpleSOAP` on a 3-layer MLP
of width 64.

The code will:
- Sweep learning rates `lr` in `[0.001, 0.003, 0.01, 0.03]` for each optimizer.
- Report the mean validation loss for each `(optimizer, lr)` pair.

In your written submission, briefly answer:
- Which learning-rate range works reasonably well for **Shampoo**?
- Which range works well for **SOAP**?
- Does preserving the Frobenius norm in SOAP appear to make learning-rate tuning
  easier or harder compared to Shampoo?

In [ ]:
# Small learning-rate comparison for Shampoo vs SOAP

# This cell sweeps over lr for SimpleShampoo and SimpleSOAP on a 3-layer MLP
# of width 64, and reports the mean validation loss for each.


def train_with_lr_opt(hiddens=[64, 64, 64], optimizer_cls=SimpleShampoo, lr=0.01):
    torch.manual_seed(4)
    np.random.seed(4)
    model = MLP(hidden_sizes=hiddens).to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optimizer_cls(model.parameters(), lr=lr)
    losses = []

    for i in range(100):
        batch_idx = np.random.randint(0, len(train_images), size=64)

        images_batch = train_images[batch_idx]
        labels_batch = train_labels[batch_idx]
        images_batch, labels_batch = images_batch.to(device), labels_batch.to(device)

        optimizer.zero_grad()
        outputs, _ = model(images_batch)
        loss = criterion(outputs, labels_batch)
        loss.backward()
        optimizer.step()

        with torch.no_grad():
            outputs_valid, _ = model(valid_images)
            valid_losses = criterion(outputs_valid, valid_labels)
            losses.append(valid_losses.item())

    return np.mean(np.array(losses)[-5:])


lrs = [0.001, 0.003, 0.01, 0.03]
optims = [SimpleShampoo, SimpleSOAP]

for opt_cls in optims:
    print(f"Results for {opt_cls.__name__}:")
    for lr in lrs:
        loss = train_with_lr_opt(hiddens=[64, 64, 64], optimizer_cls=opt_cls, lr=lr)
        print(f"  lr={lr}: mean valid loss = {loss:.3f}")